In [72]:
# _importing required libraries
import os
import re

# _external libraries
import pandas as pd

In [73]:
filepath = os.getcwd() + '/../datasets/chaii/train.csv'
chaii_df = pd.read_csv(filepath)
chaii_df.sample(3)

,id,context,question,answer_text,answer_start,language
172,eef50e78c,நானோ தொழில்நுட்பம் (Nanotechnology) அல்லது மீந...,நானோ தொழில்நுட்பத்தை கண்டுபிடித்தவர் யார்?,இரிச்சர்டு பீய்ன்மனால்,3202,tamil
351,6bb0c472d,சிங்கம் என்பது பாலூட்டி வகையைச் சேர்ந்த ஒரு கா...,பெண் சிங்கம் என்று என்ன அழைக்கப்படுகிறது?,சிம்மம்,168,tamil
740,23d8f8654,राम दास या गुरू राम दास (Punjabi: ਸ੍ਰੀ ਗੁਰੂ ਰਾ...,गुरु राम दास की पत्नी कौन थी?,बीबी भानी जी,1276,hindi


In [74]:
a = ' abc    dfe fdjk dfe'
b = ' '.join(a.split())
b

'abc dfe fdjk dfe'

In [75]:
for m in re.finditer('dfe', b):
    print('dfe found', m.start(), m.end())

dfe found 4 7
dfe found 13 16


In [76]:
def get_closest_startidx(context, answer_txt, answer_start):
    
    start_idx = answer_start
    for match in re.finditer(answer_txt, context):
        if match.start() < answer_start:
            start_idx = match.start()
        else:
            break
            
    return start_idx

In [77]:
def transform_startidx(data):
    
    context = ' '.join(data['context'].split())
    answer_txt = ' '.join(data['answer_text'].split())
    answer_start = data['answer_start']
    
    start_idx = get_closest_startidx(context, answer_txt, answer_start)
    #print(answer_start-start_idx)
    
    return [context, answer_txt, start_idx]

In [78]:
updated_data = chaii_df.apply(lambda x:transform_startidx(x), axis=1)

In [79]:
context_list = []
answer_text_list = []
answer_start_list = []

for val in updated_data:
    context_list.append(val[0])
    answer_text_list.append(val[1])
    answer_start_list.append(val[2])

In [80]:
chaii_df['context'] = context_list
chaii_df['answer_text'] = answer_text_list
chaii_df['answer_start'] = answer_start_list
chaii_df.sample(3)

,id,context,question,answer_text,answer_start,language
1058,ebe48c5e6,"इन्फोसिस लिमिटेड (BSE:, Nasdaq:) एक बहुराष्ट्र...",इंफोसिस लिमिटेड की स्थापना किस वर्ष में हुई थी?,"२ जुलाई, १९८१",473,hindi
177,36a4760ec,செப்பு (Copper) எனப்படுவது உலோக வகையைச் சேர்ந்...,செப்பு எந்த நிறத்தில் இருக்கும்?,சிவந்த,401,tamil
4,b29c82c22,சூரியக் குடும்பம் சூரியக் குடும்பம் (Solar Sys...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,582,tamil


In [81]:
def add_end_index(row):
    strt = row['answer_start']
    end = strt+len(row['answer_text'])
    ans = row['context'][strt:end]
    assert(ans==row['answer_text'])
    return end

In [82]:
chaii_df['answer_end'] = chaii_df.apply(add_end_index,axis=1)